In [4]:
# Imports here
import torch
from torch import nn,optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#from workspace_utils import active_session

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import argparse

In [31]:
#Load the data
def get_data(data_dir):
    train_dir = data_dir + '/train'
    valid_dir = data_dir + '/valid'
    test_dir = data_dir + '/test'
    
    # Define transforms for the training, validation, and testing sets
    train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

    validate_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

    test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
    # Load the datasets with ImageFolder
    train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)
    validate_datasets = datasets.ImageFolder(valid_dir, transform=validate_transforms)
    test_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)

    # Using the image datasets and the trainforms, define the dataloaders
    trainloader = torch.utils.data.DataLoader(train_datasets, batch_size=64, shuffle=True)
    validateloader = torch.utils.data.DataLoader(validate_datasets, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_datasets, batch_size=64)

    return trainloader, validateloader, testloader

In [32]:
def build_network(input_size=1024, output_size=102, hidden_units=512, drop_p=0.5,arch='densenet121'):        

    model = eval("models.{}(pretained=True)".format(arch))
    
    for params in model.parameters():
        params.requires_grad= False
    
    model.classifier = nn.Sequential(nn.Linear(input_size,hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(p=drop_p),
                                 nn.Linear(hidden_units,output_size),
                                 nn.LogSoftmax(dim=1))
    return model

In [33]:
def set_optimizer(model,learning_rate=0.003):
    return optim.Adam(model.classifier.parameters(),lr=learning_rate)

In [35]:
#Train the network
def train_network(model, trainloader, validateloader, criterion, optimizer, epochs=5, print_every=40,device='cpu'):
    steps = 0
    train_loss = 0
    for epoch in range(epochs):
        model.train()
        for images, labels in trainloader:
            steps += 1
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            log_ps = model.forward(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if steps % print_every == 0:
                validate_loss = 0
                validate_accuracy = 0
                model.eval()
                with torch.no_grad():
                    for images, labels in validateloader:
                        images, labels = images.to(device), labels.to(device)
                        log_ps = model.forward(images)
                        loss = criterion(log_ps, labels)

                        validate_loss += loss.item()

                        # Calculate accuracy
                        ps = torch.exp(log_ps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        validate_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {train_loss/print_every:.3f}.. "
                      f"Validation loss: {validate_loss/len(validateloader):.3f}.. "
                      f"Validation accuracy: {validate_accuracy/len(validateloader):.3f}")
                train_loss = 0
                model.train()
    return model

In [ ]:
def save_model(model,optimizer,save_dir='.',train_datasets,input_size=1024, output_size=102, hidden_units=512,epochs=5):
    model.class_to_idx = train_datasets.class_to_idx

    checkpoint = {'input_size': input_size,
                  'hidden_units': hidden_units
                  'output_size': output_size,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'classes_indices_map': model.class_to_idx,
                  'epoch': epochs}

    model_checkpoint=save_dir+'/checkpoint.pth'
    torch.save(checkpoint, model_checkpoint)
    return model_checkpoint

In [36]:
def main():
    # Define Command Line arguments for the script
    parser = argparse.ArgumentParser (description = "Train a new network on a dataset and save the model as a checkpoint")

    parser.add_argument ('data_dir', help = 'Provide data directory.', type = str)
    parser.add_argument ('--save_dir', help = 'Optional: Provide saving directory. Default is current directory', type = str)
    parser.add_argument ('--arch', help = 'Optional: Provide a Pretrained model. Default pretrained model is densenet121', type = str)
    parser.add_argument ('--lrn', help = 'Optional: Learning rate. Default value is 0.003', type = float)
    parser.add_argument ('--hidden_units', help = 'Optional: Hidden units in Classifier. Default value is 512', type = int)
    parser.add_argument ('--epochs', help = 'Optional: Number of epochs. Default value is 5', type = int)
    parser.add_argument ('--GPU', help = "Optional: Option to use GPU. Default is CPU", type = str)

    #setting values data loading
    args = parser.parse_args ()

if __name__ == '__main__':
    main()    

usage: ipykernel_launcher.py [-h] [--save_dir SAVE_DIR] [--arch ARCH]
                             [--lrn LRN] [--hidden_units HIDDEN_UNITS]
                             [--epochs EPOCHS] [--GPU GPU]
                             data_dir
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\balaranji\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
